In [6]:
#importing the packages
import pandas as pd
import numpy as np
import random 

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import joblib # for saving algorithm and preprocessing objects
from sklearn.linear_model import LinearRegression

In [7]:
# uploading the dataset
df = pd.read_csv('RegionalInterestByConditionOverTime.csv')
df.head()

,dma,geoCode,2004+cancer,2004+cardiovascular,2004+stroke,2004+depression,2004+rehab,2004+vaccine,2004+diarrhea,2004+obesity,...,2016+diabetes,2017+cancer,2017+cardiovascular,2017+stroke,2017+depression,2017+rehab,2017+vaccine,2017+diarrhea,2017+obesity,2017+diabetes
0,Portland-Auburn ME,500,44,6,17,39,21,31,14,29,...,81,70,37,83,64,56,76,66,47,80
1,New York NY,501,47,6,13,38,16,33,12,27,...,77,70,34,53,56,53,79,56,52,78
2,Binghamton NY,502,48,3,16,50,12,37,24,31,...,74,68,24,71,69,44,77,78,61,72
3,Macon GA,503,44,14,14,37,19,49,14,29,...,78,53,38,62,46,60,47,53,41,66
4,Philadelphia PA,504,52,7,16,41,23,36,14,30,...,80,75,35,61,62,75,84,69,56,78


In [8]:
print(df.columns)
print(df.shape)
print(df.describe())

Index(['dma', 'geoCode', '2004+cancer', '2004+cardiovascular', '2004+stroke',
       '2004+depression', '2004+rehab', '2004+vaccine', '2004+diarrhea',
       '2004+obesity',
       ...
       '2016+diabetes', '2017+cancer', '2017+cardiovascular', '2017+stroke',
       '2017+depression', '2017+rehab', '2017+vaccine', '2017+diarrhea',
       '2017+obesity', '2017+diabetes'],
      dtype='object', length=128)
(210, 128)
          geoCode  2004+cancer  2004+cardiovascular  2004+stroke  \
count  210.000000   210.000000           210.000000   210.000000   
mean   641.104762    43.904762             7.433333    17.642857   
std    100.375079     7.618944             7.909647     8.135284   
min    500.000000    27.000000             0.000000     0.000000   
25%    552.250000    40.000000             5.000000    14.000000   
50%    627.500000    43.000000             6.000000    16.000000   
75%    723.500000    47.000000             9.000000    18.000000   
max    881.000000   100.000000     

In [9]:
#IQR range
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
print(IQR)


geoCode                171.25
2004+cancer              7.00
2004+cardiovascular      4.00
2004+stroke              4.00
2004+depression         14.00
                        ...  
2017+rehab              18.00
2017+vaccine            12.00
2017+diarrhea           11.00
2017+obesity            12.75
2017+diabetes            8.00
Length: 127, dtype: float64


In [10]:
#removing Outliers
df = df[~((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR))).any(axis=1)]
df.shape

(91, 128)

In [11]:
global states
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

In [12]:
df['State_Code'] = df['dma'].apply(lambda x: x[x.rfind(" ")+1:])

In [13]:
df['State'] = df['State_Code'].apply(lambda x:states[x])
df

,dma,geoCode,2004+cancer,2004+cardiovascular,2004+stroke,2004+depression,2004+rehab,2004+vaccine,2004+diarrhea,2004+obesity,...,2017+cardiovascular,2017+stroke,2017+depression,2017+rehab,2017+vaccine,2017+diarrhea,2017+obesity,2017+diabetes,State_Code,State
0,Portland-Auburn ME,500,44,6,17,39,21,31,14,29,...,37,83,64,56,76,66,47,80,ME,Maine
1,New York NY,501,47,6,13,38,16,33,12,27,...,34,53,56,53,79,56,52,78,NY,New York
4,Philadelphia PA,504,52,7,16,41,23,36,14,30,...,35,61,62,75,84,69,56,78,PA,Pennsylvania
5,Detroit MI,505,47,6,16,45,14,31,15,27,...,38,66,61,57,73,70,45,74,MI,Michigan
6,Boston MA-Manchester NH,506,48,6,15,38,25,34,14,27,...,37,58,62,72,76,58,58,74,NH,New Hampshire
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,San Diego CA,825,42,5,15,31,14,29,11,22,...,37,57,59,37,80,61,56,76,CA,California
204,Las Vegas NV,839,33,4,16,32,16,22,16,17,...,31,55,52,52,60,65,42,70,NV,Nevada
206,Sacramento-Stockton-Modesto CA,862,40,4,17,41,12,35,13,29,...,29,64,55,36,75,66,55,76,CA,California
208,Chico-Redding CA,868,36,4,22,33,16,49,14,41,...,38,76,63,59,89,70,62,75,CA,California


In [14]:
df.shape

(91, 130)

In [15]:
temp_cols = ['cancer','cardiovascular','stroke','depression','rehab','vaccine','diarrhea','obesity','diabetes']
new_columns = ['State','Year',*temp_cols]


In [21]:
data_final = []
for _, row in df.iterrows():
    keys = list(row.keys())[2:-2]
    values = list(row[2:-2])
    for year in range(0,14):
        d = [row['State'] , year+2003 , *values[year:year+9]]
        data_final.append(d)
dd = pd.DataFrame(data_final , columns = new_columns)
dff = dd.groupby(['State','Year']).mean().reset_index()
dff

,State,Year,cancer,cardiovascular,stroke,depression,rehab,vaccine,diarrhea,obesity,diabetes
0,Alabama,2003,41.000000,7.000000,18.000000,48.000000,18.000000,29.000000,12.000000,20.000000,34.000000
1,Alabama,2004,7.000000,18.000000,48.000000,18.000000,29.000000,12.000000,20.000000,34.000000,60.000000
2,Alabama,2005,18.000000,48.000000,18.000000,29.000000,12.000000,20.000000,34.000000,60.000000,14.000000
3,Alabama,2006,48.000000,18.000000,29.000000,12.000000,20.000000,34.000000,60.000000,14.000000,46.000000
4,Alabama,2007,18.000000,29.000000,12.000000,20.000000,34.000000,60.000000,14.000000,46.000000,9.000000
...,...,...,...,...,...,...,...,...,...,...,...
569,Wisconsin,2012,63.333333,11.666667,43.000000,12.666667,19.333333,27.666667,22.666667,16.000000,20.333333
570,Wisconsin,2013,11.666667,43.000000,12.666667,19.333333,27.666667,22.666667,16.000000,20.333333,51.000000
571,Wisconsin,2014,43.000000,12.666667,19.333333,27.666667,22.666667,16.000000,20.333333,51.000000,17.333333
572,Wisconsin,2015,12.666667,19.333333,27.666667,22.666667,16.000000,20.333333,51.000000,17.333333,23.666667


In [25]:
dff['Total'] = dff['cancer'] + dff['cardiovascular'] + dff['stroke'] + dff['depression'] + dff['rehab'] + dff['vaccine'] + dff['diarrhea'] + dff['obesity'] + dff['obesity'] 
val_max = dff['Total'].max()
val_min = dff['Total'].min()
dff['Scaled_total'] = dff['Total'].apply(lambda x: (x-val_min)/(val_max-val_min))
dff

,State,Year,cancer,cardiovascular,stroke,depression,rehab,vaccine,diarrhea,obesity,diabetes,Total,Scaled_total
0,Alabama,2003,41.000000,7.000000,18.000000,48.000000,18.000000,29.000000,12.000000,20.000000,34.000000,213.000000,0.237333
1,Alabama,2004,7.000000,18.000000,48.000000,18.000000,29.000000,12.000000,20.000000,34.000000,60.000000,220.000000,0.274667
2,Alabama,2005,18.000000,48.000000,18.000000,29.000000,12.000000,20.000000,34.000000,60.000000,14.000000,299.000000,0.696000
3,Alabama,2006,48.000000,18.000000,29.000000,12.000000,20.000000,34.000000,60.000000,14.000000,46.000000,249.000000,0.429333
4,Alabama,2007,18.000000,29.000000,12.000000,20.000000,34.000000,60.000000,14.000000,46.000000,9.000000,279.000000,0.589333
...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,Wisconsin,2012,63.333333,11.666667,43.000000,12.666667,19.333333,27.666667,22.666667,16.000000,20.333333,232.333333,0.340444
570,Wisconsin,2013,11.666667,43.000000,12.666667,19.333333,27.666667,22.666667,16.000000,20.333333,51.000000,193.666667,0.134222
571,Wisconsin,2014,43.000000,12.666667,19.333333,27.666667,22.666667,16.000000,20.333333,51.000000,17.333333,263.666667,0.507556
572,Wisconsin,2015,12.666667,19.333333,27.666667,22.666667,16.000000,20.333333,51.000000,17.333333,23.666667,204.333333,0.191111


In [17]:
import pickle
pickle.dump( dd, open( "diseases_df.p", "wb" ) )

In [ ]:
import pickle
import pandas as pd
df = pickle.load( open( "diseases_df.p", "rb" ) )

In [101]:
import joblib,json
data_loaded = joblib.load('processed_data.joblib')

final_data= []

jss = data_loaded.to_json()
j = json.loads(jss)
data_loaded.shape


(799, 15)

In [99]:
zip_data = list(zip(j['State'].values(),
j['Year'].values(),
j['Total_AQI'].values()))

[('Alabama', 2000, 164.0),
 ('Alabama', 2001, 155.070407577),
 ('Alabama', 2002, 146.1408151541),
 ('Alabama', 2003, 137.2112227311),
 ('Alabama', 2004, 128.2816303081),
 ('Alabama', 2005, 119.3520378852),
 ('Alabama', 2006, 110.4224454622),
 ('Alabama', 2007, 101.4928530392),
 ('Alabama', 2008, 92.5632606163),
 ('Alabama', 2009, 83.6336681933),
 ('Alabama', 2010, 74.7040757703),
 ('Alabama', 2011, 65.7744833474),
 ('Alabama', 2012, 56.8448909244),
 ('Alabama', 2013, 47.9152985014),
 ('Alabama', 2014, 63.3651854834),
 ('Alabama', 2015, 60.537035649),
 ('Alabama', 2016, 66.2469879579),
 ('Alaska', 2000, 164.0),
 ('Alaska', 2001, 155.2949213804),
 ('Alaska', 2002, 146.5898427607),
 ('Alaska', 2003, 137.8847641411),
 ('Alaska', 2004, 129.1796855215),
 ('Alaska', 2005, 120.4746069018),
 ('Alaska', 2006, 111.7695282822),
 ('Alaska', 2007, 103.0644496626),
 ('Alaska', 2008, 94.3593710429),
 ('Alaska', 2009, 85.6542924233),
 ('Alaska', 2010, 76.9492138036),
 ('Alaska', 2011, 68.244135184),
 (

In [ ]:
state_data = {}
for state, year, score in zip_data:
    state_data['States'] = state
    state_data['Year'] = year
    state_data[year] = score
    